In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
import numpy as np
from requests_html import HTMLSession
import psycopg2 as pg2

In [2]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'
headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
r = requests.get(main_link, headers=headers)
r

<Response [200]>

## Main Idea
### scrape all main_link + sub_links where sub_links=data%20analyst,data%20scientist,data%20engineer

In [3]:
soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
max_page = soup.find('li',class_='ant-pagination-next').previous_sibling.text
max_pages = int(max_page)

In [5]:
max_pages

160

### working out how to turn pages:

In [6]:
page_link_list=[]
for i in range(0,max_pages+1):
    page_link_list.append('https://www.kariera.gr/en/jobs?title=&page={}&limit=20'.format(i))

## getting all sub-links from all links

In [7]:
link_list=[]
session = HTMLSession()
for i in tqdm(page_link_list):
    for y in session.get(i).html.links:
        if '/en/jobs/' in y:
            link_list.append(y)

  0%|          | 0/161 [00:00<?, ?it/s]

In [11]:
len(link_list)

3122

In [12]:
final_list=[]
main = 'https://www.kariera.gr/'
for i in link_list:
    final_list.append(main + i)

In [13]:
test_link = final_list[1]

In [14]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
r = requests.get(test_link, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

### job title

In [15]:
soup.find('div',class_='h4 UGhAL2CE').text

'Λογιστής / ρια - Φοροτεχνικός'

### company

In [16]:
soup.find('a',class_='__8_RJM1pE').text

'CERTUS ΕΕ'

### location

In [17]:
soup.findAll('a',class_='V_oVSppD main-body-text')[0].text

'Athina'

### job occupation

In [18]:
getattr(soup.findAll('a',class_='V_oVSppD main-body-text')[3],'text',None)

'Finance'

In [19]:
soup.findAll('a',class_='V_oVSppD main-body-text')[3].text

'Finance'

### level

In [20]:
soup.findAll('a',class_='V_oVSppD main-body-text')[1].text

'Mid level'

### job type

In [21]:
soup.findAll('a',class_='V_oVSppD main-body-text')[2].text

'Full-time'

### content

In [22]:
result = soup.findAll('div',class_='__2DY4wJ3z hi8OBmAZ')
a=''
for res in result:
    a= (res.get_text(strip=True,separator=' '))
    a = a.replace('\u202f','').replace('\xa0','').replace('/','')

# building the loop

In [23]:
df2 = pd.DataFrame()

In [24]:
for i in tqdm(final_list):
    link = i
    unique_id = i[-5:]
    a=''
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    if soup.find('div',class_='h4 UGhAL2CE') is not None:
        x = soup.find('div',class_='h4 UGhAL2CE').text
        if ('Data Analyst' in x) | ('Data Scientist' in x) | ('Data Engineer' in x): 
            try:
                job_title = soup.find('div',class_='h4 UGhAL2CE').text
            except:
                job_title = np.nan
            try:
                company = soup.find('a',class_='__8_RJM1pE').text
            except:
                company = np.nan
            try:
                location = soup.findAll('a',class_='V_oVSppD main-body-text')[0].text
            except:
                location = np.nan
            try:
                job_occupation = soup.findAll('a',class_='V_oVSppD main-body-text')[3].text
            except:
                job_occupation = np.nan
            try:
                level = soup.findAll('a',class_='V_oVSppD main-body-text')[1].text
            except:
                level = np.nan
            try:
                job_type = soup.findAll('a',class_='V_oVSppD main-body-text')[2].text
            except:
                job_type = np.nan
            try:
                result = soup.findAll('div',class_='__2DY4wJ3z hi8OBmAZ')
                for i in result:
                    a= i.get_text(strip=True,separator=' ')
                    a = a.replace('\u202f','').replace('\xa0','').replace('/','')
            except:
                content = np.nan
            temp = pd.DataFrame([{
                    'job_id' : unique_id,
                    'job_title' : job_title,
                    'company' : company,
                    'location': location,
                    'job_occupation' : job_occupation,
                    'level' : level,
                    'job_type' : job_type,
                    'content' : a,
                    'date_scraped' : datetime.now().strftime("%Y-%m-%d %H:%M")
                }])
            df2 = pd.concat([df2, temp], ignore_index=True)

  0%|          | 0/3122 [00:00<?, ?it/s]

In [25]:
name1 = 'content' + '_' + str(datetime.now().strftime("%d-%m-%Y") + '.csv')

In [26]:
df2.to_csv(name1, index=False)

## Connect to postgres

In [27]:
conn = pg2.connect(database='Jobs',user='postgres',password='no_pwsd_here')

In [28]:
cur = conn.cursor()

In [29]:
def create_table(cur):
    create_table_command = (""" 
    CREATE TABLE IF NOT EXISTS jobs (
    job_id int primary key NOT NULL,
   job_title text NOT NULL,
   company text,
   location text,
   job_occupation text,
   level text,
   job_type text,
   content text,
   date_scraped timestamp NOT NULL
    );
    """)
    
    cur.execute(create_table_command)

In [30]:
create_table(cur)

In [31]:
def check_if_job_exists(cur,job_id):
    query = (""" select job_id from jobs where job_id = %s """)
    cur.execute(query,(job_id,))
    
    return cur.fetchone() is not None

In [32]:
def update_db(cur, df):

    l = []
    for row in df.iterrows():
        if check_if_job_exists(cur, row[1][0]):
            pass
        else:
            l.append(row[1][0])

    return l

# Create dataframe with new jobs

In [33]:
new_jobs = update_db(cur,df2)

In [34]:
new_jobs = df2[df2['job_id'].apply(lambda x : x in new_jobs)]

In [35]:
new_jobs

,job_id,job_title,company,location,job_occupation,level,job_type,content,date_scraped
0,41993,Data Engineer,Executive Level,Athina,IT,Mid level,Full-time,Data Engineer Responsibilities Identify data s...,2023-01-09 14:21


## Push new jobs to postgress

In [36]:
def insert_into_table(cur,job_id, job_title, company, location, job_occupation, level,job_type, content, date_scraped):
    insert_new_jobs = (
        """ INSERT INTO jobs (job_id, job_title, company, location, job_occupation, level,
           job_type, content, date_scraped)
            VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s);""")

    job_to_insert = (job_id, job_title, company, location, job_occupation, level,
                     job_type, content, date_scraped)

    cur.execute(insert_new_jobs,job_to_insert)

In [37]:
def push_from_df_to_db(cur, df):
    for i, row in df.iterrows():
        insert_into_table(cur, row['job_id'], row['job_title'], row['company'],
                          row['location'], row['job_occupation'], row['level'],
                          row['job_type'], row['content'], row['date_scraped'])

In [38]:
push_from_df_to_db(cur, new_jobs)

In [39]:
conn.commit()

In [40]:
conn.close()

# Fetch jobs table and save to a df

In [41]:
conn = pg2.connect(database='Jobs', user='postgres', password='new_password')

with conn:

    cur = conn.cursor()
    cur.execute("SELECT * FROM jobs")

    rows = cur.fetchall()

df = pd.DataFrame(rows).rename(
    columns={
        0: 'job_id',
        1: 'job_title',
        2: 'company',
        3: 'location',
        4: 'job_occupation',
        5: 'level',
        6: 'job_type',
        7: 'content',
        8: 'date_scraped'
    })

conn.close()

In [42]:
df.head()

,job_id,job_title,company,location,job_occupation,level,job_type,content,date_scraped
0,38344,Data Analyst,Mellon Technologies,Pireas,IT,Mid level,Full-time,Description Responsible for the quality and co...,2022-12-17 01:39:00
1,37348,Data Engineer - Patras,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 146677 At EY, you’ll have the ...",2022-12-17 01:47:00
2,37357,Junior Data Engineer,Ernst and Young,Athina,IT,Mid level,Full-time,Requisition ID: 127061 Junior Data Engineer At...,2022-12-17 01:47:00
3,37349,Data & Analytics Manager - Data Engineering,Ernst and Young,Athina,IT,Senior,Full-time,"Requisition ID: 814902 At EY, you’ll have the ...",2022-12-17 01:47:00
4,37345,Data Scientist - Patras,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 147180 At EY, you’ll have the ...",2022-12-17 01:47:00


## Cleaning company names

In [43]:
df['company'] = df['company'].apply(lambda x : x.replace('\t','').strip())

## Categorizing jobs

In [44]:
df['categories'] = df['job_title'].apply(lambda x: 'Analytics' if 'Analyst' in x else
                      ('Engineering' if 'Engineer' in x else 'Science'))

## Cleaning job_title

In [45]:
# removing spaces
df['job_title'] = df['job_title'].apply(lambda x : x.strip())

In [46]:
# removing locations from job titles
# it assumes that always all locations will be preceded by ' - '
def clean_city_from_job(job,city):
    titles = []
    for i,j in zip(job,city):
        if j in i:
            title = i[:-(3+len(j))]
            titles.append(title)
        else:
            title = i
            titles.append(title)
    return pd.DataFrame(titles).rename(columns={0:'job_title'})

In [47]:
df = clean_city_from_job(df['job_title'], df['location']).merge(
    df, left_index=True,
    right_index=True).drop('job_title_y',
                           axis=1).rename(columns={'job_title_x': 'job_title'})

In [48]:
df.head()

,job_title,job_id,company,location,job_occupation,level,job_type,content,date_scraped,categories
0,Data Analyst,38344,Mellon Technologies,Pireas,IT,Mid level,Full-time,Description Responsible for the quality and co...,2022-12-17 01:39:00,Analytics
1,Data Engineer,37348,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 146677 At EY, you’ll have the ...",2022-12-17 01:47:00,Engineering
2,Junior Data Engineer,37357,Ernst and Young,Athina,IT,Mid level,Full-time,Requisition ID: 127061 Junior Data Engineer At...,2022-12-17 01:47:00,Engineering
3,Data & Analytics Manager - Data Engineering,37349,Ernst and Young,Athina,IT,Senior,Full-time,"Requisition ID: 814902 At EY, you’ll have the ...",2022-12-17 01:47:00,Engineering
4,Data Scientist,37345,Ernst and Young,Patras,IT,Mid level,Full-time,"Requisition ID: 147180 At EY, you’ll have the ...",2022-12-17 01:47:00,Science


## Cleaning content

In [49]:
df['content'] = df['content'].apply(
    lambda x: x.replace('Description', '').replace('Company', '').replace(
        'ΠΕΡΙΓΡΑΦΗ ΘΕΣΗΣ ΕΡΓΑΣΙΑΣ', '').replace('Job', '').replace(
            '’s', '').replace('Purpose', '').strip())

In [50]:
import re

In [51]:
df['content'] = df['content'].apply(lambda x : re.sub(r'Requisition ID: \d+', '', x).strip())

## Geolocate locations

In [52]:
from geopy import geocoders
import swifter
gn = geocoders.GeoNames('asterios')

In [53]:
df['geo'] = df['location'].swifter.apply(lambda x : gn.geocode(x)).apply(lambda loc: tuple(loc.point) if loc else None)

Pandas Apply:   0%|          | 0/28 [00:00<?, ?it/s]

In [54]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['geo'].tolist(), index=df.index)

In [55]:
df.head(2)

,job_title,job_id,company,location,job_occupation,level,job_type,content,date_scraped,categories,geo,latitude,longitude,altitude
0,Data Analyst,38344,Mellon Technologies,Pireas,IT,Mid level,Full-time,Responsible for the quality and completeness o...,2022-12-17 01:39:00,Analytics,"(37.94203, 23.64619, 0.0)",37.94203,23.64619,0.0
1,Data Engineer,37348,Ernst and Young,Patras,IT,Mid level,Full-time,"At EY, you’ll have the chance to build a caree...",2022-12-17 01:47:00,Engineering,"(38.24444, 21.73444, 0.0)",38.24444,21.73444,0.0


### At this point we are done with the cleaning next up is droping unwanted columns and exporting  the csv for the main app

In [58]:
df

,job_title,company,location,level,job_type,content,categories,latitude,longitude
0,Data Analyst,Mellon Technologies,Pireas,Mid level,Full-time,Responsible for the quality and completeness o...,Analytics,37.94203,23.64619
1,Data Engineer,Ernst and Young,Patras,Mid level,Full-time,"At EY, you’ll have the chance to build a caree...",Engineering,38.24444,21.73444
2,Junior Data Engineer,Ernst and Young,Athina,Mid level,Full-time,"Junior Data Engineer At EY, you’ll have the ch...",Engineering,37.98376,23.72784
3,Data & Analytics Manager - Data Engineering,Ernst and Young,Athina,Senior,Full-time,"At EY, you’ll have the chance to build a caree...",Engineering,37.98376,23.72784
4,Data Scientist,Ernst and Young,Patras,Mid level,Full-time,"At EY, you’ll have the chance to build a caree...",Science,38.24444,21.73444
5,Senior Data Engineer,EUROPEAN DYNAMICS,Marousi,Senior,Full-time,We currently have vacancy position for a Senio...,Engineering,38.05000,23.80000
6,Data Engineer,PeopleCert,Athina,Senior,Full-time,Are you interested in working with a leading e...,Engineering,37.98376,23.72784
7,Data Engineer and Reporting Analyst,AEGEAN AIRLINES,Spata,Entry/Junior,Full-time,Data Engineer and Reporting Analyst Division: ...,Analytics,37.96163,23.91514
8,Data Analyst,Tech4sGroup,Limassol,Mid level,Full-time,"Tech4S Group, is a fast growing and innovative...",Analytics,34.68406,33.03794
9,Python Developer and Data Scientist,NaN,Pireas,Mid level,Full-time,A financial services company is looking for a ...,Science,37.94203,23.64619


In [56]:
df.drop(['job_id','job_occupation','date_scraped','geo','altitude'],axis=1,inplace=True)

In [57]:
df.to_csv('job_board.csv',index=False)